In [1]:
from urllib.request import urlopen
import pubchempy as pcp
import numpy as np
import pandas as pd
import requests
import pandas as pd
from chembl_structure_pipeline.exclude_flag import exclude_flag
from chembl_structure_pipeline.standardizer import (
    get_parent_mol, update_mol_valences, kekulize_mol, flatten_tartrate_mol,
    normalize_mol, uncharge_mol, cleanup_drawing_mol)
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdmolops

####### SOURCES:
# https://stackoverflow.com/questions/54930121/converting-molecule-name-to-smiles

In [2]:
def get_smiles_df(identifiers):
    smiles_df = pd.DataFrame(columns = ['Name', 'Smiles'])
    for x in identifiers :
        try:
            url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/' + x + '/property/CanonicalSMILES/TXT'
            smiles = requests.get(url).text.rstrip()
            if('NotFound' in smiles):
                print(x, " not found")
            else: 
                smiles_df = smiles_df.append({'Name' : x, 'Smiles' : smiles}, ignore_index = True)
        except: 
            print("boo ", x)
    return (smiles_df)

######## GET CID
def get_cid(df):
    df['CID'] = np.nan
    for idx, row in df.iterrows():
            if pd.isna(row["CID"]):
                try:
                    compound = pcp.get_compounds(identifier=row["Smiles"],  namespace="smiles")
                    df.loc[idx, "CID"] = compound[0].cid
                    if compound[0].cid is not None:
                        counter += 1

                except:
                    pass
    return df

###### Adding isomeric smiles
def get_isomeric_smiles(df):
    if "isomeric_smiles" not in df.columns.to_list():
        df["isomeric_smiles"] = ""
    if "iupac_name" not in df.columns.tolist():
        df["iupac_name"] = ""
    if "smiles_result" not in df.columns.tolist():
        df["smiles_result"] = ""

    for idx, row in df.iterrows():
            try:
                if not pd.isna(row["CID"]):
                    # compound = pcp.Compound.from_cid(int(row["CID"]))
                    url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{}/property/isomericsmiles/txt".format(int(row["CID"]))
                    isomeric_smiles = urlopen(url).read().decode('utf8').strip()
                    # add isomeric smiles
                    df.loc[idx, "isomeric_smiles"] = isomeric_smiles
                    # add iupac names
                    url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{}/property/IUPACName/txt".format(int(row["CID"]))
                    df.loc[idx, "iupac_name"] = urlopen(url).read().decode('utf8').strip()

                    # if CID is none, isomeric smiles is none
                    df.loc[idx, "smiles_result"] = isomeric_smiles
                else:
                    df.loc[idx, "smiles_result"] = row["smiles"]
            except:
                pass
    return df

def get_smiles_result(df):
    for idx, row in df.iterrows():
            if pd.isna(row["smiles_result"]):
                df.loc[idx, "smiles_result"] = row["smiles"]
    return df

In [3]:
metal_atoms = {3, 4, 11, 12, 13, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,
               29, 30, 31, 32, 34, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46,
               47, 48, 49, 50, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
               66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80,
               81, 82, 83, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98,
               99, 100, 101, 102, 103}
heavy_atoms = {21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34,
               37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
               50, 51, 52, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,
               65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
               78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90,
               91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102,
               103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
               114, 115, 116, 117, 118}

unwanted_atom_dict = {
    "metal_atoms": metal_atoms,
    "heavy_atoms": heavy_atoms,
}

In [4]:
class CleanMoleculesFromDataFrame:
    def __init__(self,
                 df,
                 output_fname="2_bbb_all_complete_CID_out_smiles_fixed.xlsx",
                 unwanted_atoms="heavy_atoms",
                 ):
        """
        Clean molecules from a single SDF file with multiple molecules.
        Notes
        -----
        1. Now we only support SDF file format only. For MOL2 format, one may refer to
        https://chem-workflows.com/articles/2020/03/23/building-a-multi-molecule-mol2-reader-for
        -rdkit-v2/, which remains to be tested.
        2. When using SaltRemover from rdkit, all the hydrogen should not be imported, but should be
        ignored. Otherwise, it will not work.
        """
#         self.input_fname = input_fname
        self.output_fname = output_fname
        self.unwanted_atoms = unwanted_atoms

        self.df = df

    @staticmethod
    def skip_problematic_molecules(df,
                                   second_check=False,
                                   ignore_none=True,
                                   ignore_empty_mol=False):
        """Skip problematic molecules."""
        if second_check:
            smi_label = "smiles_fixed_rdkit"
        else:
            smi_label = "smiles_result"

        for idx, row in df.iterrows():
            try:
                mol = Chem.MolFromSmiles(row[smi_label])
                if ignore_none is True and ignore_empty_mol is False:
                    if mol is not None:
                        df.loc[idx, "smiles_fixed_rdkit"] = Chem.MolToSmiles(mol, canonical=True)
                    else:
                        df.loc[idx, "smiles_fixed_rdkit"] = ""

                if ignore_none is False and ignore_empty_mol is True:
                    if mol.GetNumAtoms() != 0:
                        df.loc[idx, "smiles_fixed_rdkit"] = Chem.MolToSmiles(mol, canonical=True)
                    else:
                        df.loc[idx, "smiles_fixed_rdkit"] = ""

                if ignore_none is True and ignore_empty_mol is True:
                    if mol is not None and mol.GetNumAtoms() != 0:
                        df.loc[idx, "smiles_fixed_rdkit"] = Chem.MolToSmiles(mol, canonical=True)
                    else:
                        df.loc[idx, "smiles_fixed_rdkit"] = ""

            except:
                df.loc[idx, "smiles_fixed_rdkit"] = ""
                print(row["NO."])


        # df = df.dropna(axis=0, subset=["smiles_fixed_rdkit"], inplace=False)
        df = df[df["smiles_fixed_rdkit"] != ""]
        df = df.reset_index(drop=True)
        return df

    @staticmethod
    def filter_restricted_atoms(df, unwanted_atoms="heavy_atoms"):
        """Filter by unwanted atom set."""
        unwanted_atom_set = unwanted_atom_dict.get(unwanted_atoms)

        metal_count = 0
        for idx, row in df.iterrows():
            if row["smiles_fixed_rdkit"] != "":
                try:
                    mol = Chem.MolFromSmiles(row["smiles_fixed_rdkit"])
                    atom_list = [atom.GetAtomicNum() for atom in mol.GetAtoms()]

                    if not set(atom_list).isdisjoint(unwanted_atom_set):
                        df.loc[idx, "smiles_fixed_rdkit"] = ""
                        metal_count += 1
                except AttributeError:
                    continue

        df = df[df["smiles_fixed_rdkit"] != ""]
        df = df.reset_index(drop=True)
        return df

    #######################################################
    # # get_parent_mol() already implemented this functionality
    # @staticmethod
    # def keep_largest_fragment(df):
    #     """Only keep the largest molecular fragment"""

    #     for idx, row in df.iterrows():
    #         if row["smiles_fixed_rdkit"] != "":
    #             mol = Chem.MolFromSmiles(row["smiles_fixed_rdkit"])
    #             mols = rdmolops.GetMolFrags(mol, asMols=True)
    #             mol_new = max(mols, default=mol, key=lambda m: m.GetNumAtoms())
    #             df.loc[idx, "smiles_fixed_rdkit"] = Chem.MolToSmiles(mol_new,
    #                                                                  canonical=True)

    #     return df

    @staticmethod
    def handle_solvents_salts(df,
                              neutralize=False,
                              check_exclusion=True,
                              verbose=False):
        """"""
        # get_parent_mol() function
        # can lead to an error of converting smiles
        # if input smiles to build a molecule is
        # c1(oc([CH2+]SCCN\C(=C\[N+](=O)[O-])NC)cc1)CN(C)C
        # output molecule smiles would be
        # CN/C(=C\\[N+](=O)[O-])NCCS[CH3]c1ccc(CN(C)C)o1
        # however, loading CN/C(=C\\[N+](=O)[O-])NCCS[CH3]c1ccc(CN(C)C)o1 cannot build a molecule
        # df['column'] = df['column'].fillna(value)
        df["smiles_fixed_rdkit"] = df["smiles_fixed_rdkit"].fillna("")

        for idx, row in df.iterrows():
            if row["smiles_fixed_rdkit"] != "":
                mol = Chem.MolFromSmiles(row["smiles_fixed_rdkit"])
                mol_new, _ = get_parent_mol(mol,
                                            neutralize=neutralize,
                                            check_exclusion=check_exclusion,
                                            verbose=verbose)

                df.loc[idx, "smiles_fixed_rdkit"] = Chem.MolToSmiles(mol_new,
                                                                     canonical=True)

        df = df[df["smiles_fixed_rdkit"] != ""]
        df = df.reset_index(drop=True)
        return df

    @staticmethod
    def standardize_molecules(df, check_exclusion=True):
        """Standardize molecules with ChEMBL Structure Pipeline."""
        for idx, row in df.iterrows():
            if row["smiles_fixed_rdkit"] != "":
                mol = Chem.MolFromSmiles(row["smiles_fixed_rdkit"])

                if check_exclusion:
                    # Rules to exclude structures.
                    # Metallic or non metallic with more than 7 boron atoms will be excluded
                    # due to problems when depicting borane compounds.
                    exclude = exclude_flag(mol, includeRDKitSanitization=False)
                else:
                    exclude = False

                if not exclude:
                    mol = update_mol_valences(mol)
                    # removes all SubstanceGroups from a molecule (if any)
                    # mol = remove_sgroups_from_mol(mol)

                    mol = kekulize_mol(mol)

                    # mol = remove_hs_from_mol(mol)

                    # https://github.com/chembl/ChEMBL_Structure_Pipeline/blob
                    # /badad4f0432e7139adaa2ddac9e1c081be1e90fd/chembl_structure_pipeline
                    # /standardizer.py#L38-62
                    mol = normalize_mol(mol)
                    mol = uncharge_mol(mol)
                    # mol = flatten_tartrate_mol(mol)
                    # cleanup_drawing_mol() only works for 2D molecules
                    # mol = cleanup_drawing_mol(mol)

                df.loc[idx, "smiles_fixed_rdkit"] = Chem.MolToSmiles(mol, canonical=True)

        df = df[df["smiles_fixed_rdkit"] != ""]
        df = df.reset_index(drop=True)
        return df

    @staticmethod
    def neutralize(df):
        # Neutralizing molecules
        for idx, row in df.iterrows():
            if row["smiles_fixed_rdkit"] != "":
                smi_new, _ = neutralize_charges(smiles=row["smiles_fixed_rdkit"], reactions=None)
                df.loc[idx, "smiles_fixed_rdkit"] = smi_new

        return df

    def __call__(self):
        # filter molecules by if readable with RdKit and if unwanted atoms in molecules

        df = self.df
        df["smiles_fixed_rdkit"] = ""
        df = self.skip_problematic_molecules(df,
                                             ignore_none=True,
                                             second_check=False,
                                             ignore_empty_mol=False)

        # df = self.filter_restricted_atoms(df,
        #                                   self.unwanted_atoms)

        # keep the largest fragment
        # df = self.keep_largest_fragment(df)

        # get parent molecule by removing isotopes, solvents and salts
        # molecule neutralization is disabled for now as it can lead to an error
        df = self.handle_solvents_salts(df,
                                        neutralize=False,
                                        check_exclusion=True,
                                        verbose=False)

        df = self.neutralize(df)

        df = self.filter_restricted_atoms(df,
                                          self.unwanted_atoms)

        # standardize molecules including uncharge molecules
        df = self.standardize_molecules(df, check_exclusion=True)

        # skip problematic molecules
        df = self.skip_problematic_molecules(df,
                                             ignore_none=True,
                                             second_check=True,
                                             ignore_empty_mol=True)
        # sort result with order: logBB, CID, ...
#         df = df.sort_values(by=["logBB", "smiles_fixed_rdkit", "CID"])
        # save result
        df = df.sort_values(by=["smiles_fixed_rdkit", "CID"])
#         df.to_excel(self.output_fname, index=None, engine="openpyxl")
        return df

_reactions = None


def neutralize_charges(smiles, reactions=None):
    """ contribution from Hans de Winter """
    global _reactions
    if reactions is None:
        if _reactions is None:
            _reactions = _InitialiseNeutralisationReactions()
        reactions = _reactions
    mol = Chem.MolFromSmiles(smiles)
    replaced = False
    for i, (reactant, product) in enumerate(reactions):
        while mol.HasSubstructMatch(reactant):
            replaced = True
            rms = AllChem.ReplaceSubstructs(mol, reactant, product)
            mol = rms[0]
    if replaced:
        return Chem.MolToSmiles(mol, True), True
    else:
        return smiles, False


def _InitialiseNeutralisationReactions():
    """Contribution from Hans de Winter."""
    patts = (
        # Imidazoles
        ('[n+;H]', 'n'),
        # Amines
        ('[N+;!H0]', 'N'),
        # Carboxylic acids and alcohols
        ('[$([O-]);!$([O-][#7])]', 'O'),
        # Thiols
        ('[S-;X1]', 'S'),
        # Sulfonamides
        ('[$([N-;X2]S(=O)=O)]', 'N'),
        # Enamines
        ('[$([N-;X2][C,N]=C)]', 'N'),
        # Tetrazoles
        ('[n-]', '[nH]'),
        # Sulfoxides
        ('[$([S-]=O)]', 'S'),
        # Amides
        ('[$([N-]C=O)]', 'N'),
        )
    return [(Chem.MolFromSmarts(x), Chem.MolFromSmiles(y, False)) for x, y in patts]


def get_inchi(clean): 
    for idx, row in clean.iterrows():
            try:
                mol = Chem.MolFromSmiles(row["smiles_fixed_rdkit"])
                if mol is not None:
                    inchi_value = Chem.MolToInchi(mol)
                    clean.loc[idx, "Inchi"] = inchi_value
            except:
    #             clean.loc[idx, "comments"] = row["comments"] + "|error of generating inchi"
                print('error of generating inchi', mol)
    return clean

# Pipeline

In [5]:
Videodef pipeline(path):
    avd = pd.read_csv(path, sep='\t')
    l = avd['Drug Name'].values
    df = get_smiles_df(l)
    df = get_cid(df)
    df = get_isomeric_smiles(df)
    df = get_smiles_result(df)
    cleaning_molecules = CleanMoleculesFromDataFrame(df)
    clean = cleaning_molecules()
    df = get_inchi(clean)
    return df

In [6]:
b3db_path = '/home/shan/Downloads/B3DB_classification_extended.tsv'
avd_path = '/home/shan/Downloads/antiviral_drugs.tsv'

b3db_inch = pd.read_csv(b3db_path, sep='\t').Inchi.values.tolist()
b3db_inch = set(b3db_inch)

df = pipeline(avd_path)
ad_inch = set(df.Inchi.values.tolist())
len(ad_inch.intersection(b3db_inch))

/home/shan/anaconda3/envs/my-rdkit-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


36

In [7]:
b3db_df = pd.read_csv(b3db_path, sep='\t')
b3db_df

,NO.,compound_name,IUPAC_name,SMILES,CID,logBB,BBB+/BBB-,Inchi,threshold,reference,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,1,sulphasalazine,2-hydroxy-5-[[4-(pyridin-2-ylsulfamoyl)phenyl]...,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,5339.0,-2.69,BBB-,InChI=1S/C18H14N4O5S/c23-16-9-6-13(11-15(16)18...,NaN,R2|R2|R25|R46|,...,10.171643,63.201012,398.068491,9.477821,2428,42,144,165,9.590278,6.097222
1,2,moxalactam,7-[[2-carboxy-2-(4-hydroxyphenyl)acetyl]amino]...,COC1(NC(=O)C(C(=O)O)c2ccc(O)cc2)C(=O)N2C(C(=O)...,3889.0,-2.52,BBB-,InChI=1S/C20H20N6O9S/c1-25-19(22-23-24-25)36-8...,NaN,R25|,...,10.893010,87.033695,520.101247,9.287522,4114,62,194,237,13.756944,7.916667
2,3,clioquinol,5-chloro-7-iodoquinolin-8-ol,Oc1c(I)cc(Cl)c2cccnc12,2788.0,-2.40,BBB-,InChI=1S/C9H5ClINO/c10-6-4-7(11)9(13)8-5(6)2-1...,NaN,R18|R26|R27|,...,9.618402,44.825548,304.910439,16.939469,218,21,68,81,4.805556,2.861111
3,4,bbcpd11 (cimetidine analog) (y-g13),2-[2-[(3-bromopyridin-2-yl)methylsulfanyl]ethy...,CCNC(=NCCSCc1ncccc1Br)NC#N,14022517.0,-2.15,BBB-,InChI=1S/C12H16BrN5S/c1-2-15-12(18-9-14)17-6-7...,NaN,R2|R2|R8|R40|R2|R2|R2|R2|R18|R21|R25|R25|R26|R...,...,9.201199,50.610337,341.030979,9.743742,898,22,82,88,6.583333,4.694444
4,5,schembl614298,"(2s,3s,4s,5r)-6-[[(4r,4ar,7s,7ar,12bs)-7-hydro...",CN1CC[C@]23c4c5ccc(OC6O[C@H](C(=O)O)[C@@H](O)[...,18595497.0,-2.15,BBB-,InChI=1S/C23H27NO9/c1-24-7-6-23-10-3-4-12(25)2...,NaN,R25|,...,10.992336,85.350582,461.168581,7.686143,2850,74,198,254,11.229167,6.847222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7802,7803,antazoline,"n-benzyl-n-(4,5-dihydro-1h-imidazol-2-ylmethyl...",c1ccc(CN(CC2=NCCN2)c2ccccc2)cc1,2200.0,NaN,BBB-,InChI=1S/C17H19N3/c1-3-7-15(8-4-1)13-20(14-17-...,NaN,R14|R23|R50|,...,9.582249,66.060685,265.157898,6.798920,829,24,100,113,4.444444,4.527778
7803,7804,emedastine,"1-(2-ethoxyethyl)-2-(4-methyl-1,4-diazepan-1-y...",CCOCCn1c(N2CCCN(C)CC2)nc2ccccc21,3219.0,NaN,BBB+,InChI=1S/C17H26N4O/c1-3-22-14-13-21-16-8-5-4-7...,NaN,R15|R23|R50|,...,9.863134,70.340391,302.210661,6.296055,1041,35,112,131,6.166667,5.027778
7804,7805,ketotifen,2-(1-methylpiperidin-4-ylidene)-6-thiatricyclo...,CN1CCC(=C2c3ccccc3CC(=O)c3sccc32)CC1,3827.0,NaN,BBB+,InChI=1S/C19H19NOS/c1-20-9-6-13(7-10-20)18-15-...,NaN,R15|R27|R50|,...,10.124268,70.590222,309.118735,7.539481,917,40,122,148,5.888889,4.750000
7805,7806,milrinone,6-methyl-2-oxo-5-pyridin-4-yl-1h-pyridine-3-ca...,Cc1[nH]c(=O)c(C#N)cc1-c1ccncc1,4197.0,NaN,BBB-,InChI=1S/C12H9N3O/c1-8-11(9-2-4-14-5-3-9)6-10(...,NaN,R15|R23|R27|,...,9.618801,48.335122,211.074562,8.442982,425,24,80,93,5.555556,3.666667


In [12]:
avd_b3db = pd.merge(b3db_df, df['Inchi'], how='inner', on=['Inchi'])
avd_b3db

,NO.,compound_name,IUPAC_name,SMILES,CID,logBB,BBB+/BBB-,Inchi,threshold,reference,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,69,didanosine,9-[5-(hydroxymethyl)oxolan-2-yl]-1h-purin-6-one,O=c1[nH]cnc2c1ncn2C1CCC(CO)O1,135422442.0,-1.30,BBB-,InChI=1S/C10H12N4O3/c15-3-6-1-2-7(17-6)14-5-13...,NaN,R25|R35|R49|R2|R2|R2|R2|R3|R12|R27|R36|R38|R41...,...,9.794732,65.913447,236.090940,8.141067,502,25,92,111,4.916667,3.777778
1,101,nsc606170,4-amino-1-[5-(hydroxymethyl)oxolan-2-yl]pyrimi...,Nc1ccn(C2CCC(CO)O2)c(=O)n1,5718.0,-1.20,BBB-,InChI=1S/C9H13N3O3/c10-7-3-4-12(9(14)11-7)8-2-...,NaN,R4|R27|R43|R40|,...,9.487442,60.503043,211.095691,7.539132,369,20,76,88,5.305556,3.388889
2,125,127779-20-8,"n-[4-[3-(tert-butylcarbamoyl)-3,4,4a,5,6,7,8,8...",CC(C)(C)NC(=O)C1CC2CCCCC2CN1CC(O)C(Cc1ccccc1)N...,3579812.0,-1.03,BBB-,"InChI=1S/C38H50N6O5/c1-38(2,3)43-37(49)32-20-2...",NaN,R3|R25|R38|R41|R47|R40|R27|R2|R2|R4|,...,10.754429,87.072869,670.384269,6.771558,9512,77,256,295,16.729167,10.569444
3,151,nelfinavir mesylate,n-tert-butyl-2-[2-hydroxy-3-[(3-hydroxy-2-meth...,Cc1c(O)cccc1C(=O)NC(CSc1ccccc1)C(O)CN1CC2CCCCC...,4451.0,-0.93,BBB+,InChI=1S/C32H45N3O4S/c1-21-25(15-10-16-28(21)3...,NaN,R2|R2|R40|R25|R27|R47|R27|R4|,...,10.560671,77.066148,567.313078,6.674272,5398,63,208,240,14.145833,8.652778
4,173,aciclovir,2-amino-9-(2-hydroxyethoxymethyl)-3h-purin-6-one,Nc1nc(=O)c2ncn(COCCO)c2[nH]1,135398513.0,-0.80,BBB+,InChI=1S/C8H11N5O3/c9-8-11-6-5(7(15)12-8)10-3-...,NaN,R21|R25|R27|R43|R46|R47|R8|R2|R2|R27|R40|R50|R...,...,9.592537,62.224603,225.086189,8.336526,453,22,80,93,5.555556,3.666667
5,195,indinavir,"1-[4-benzyl-2-hydroxy-5-[(2-hydroxy-2,3-dihydr...",CC(C)(C)NC(=O)C1CN(Cc2cccnc2)CCN1CC(O)CC(Cc1cc...,3706.0,-0.74,BBB+,"InChI=1S/C36H47N5O4/c1-36(2,3)39-35(45)31-24-4...",NaN,R46|R2|R2|R12|R25|R35|R41|R3|R38|R40|R27|R36|R...,...,10.639910,96.869022,613.362805,6.666987,7788,68,236,272,14.506944,9.708333
6,200,zidovudine,1-[4-azido-5-(hydroxymethyl)oxolan-2-yl]-5-met...,Cc1cn(C2CC(N=[N+]=[N-])C(CO)O2)c(=O)[nH]c1=O,5726.0,-0.72,BBB+,InChI=1S/C10H13N5O4/c1-5-3-15(10(18)12-9(5)17)...,NaN,R47|R36|R2|R2|R8|R21|R27|R40|R3|R4|R38|R41|R43|,...,9.805930,66.192889,267.096754,8.346774,701,29,96,113,7.527778,4.361111
7,210,n-[3-hydroxy-5-[[3-methyl-2-[[[methyl-[(2-prop...,"1,3-thiazol-5-ylmethyl n-[3-hydroxy-5-[[3-meth...",CC(C)c1nc(CN(C)C(=O)NC(C(=O)NC(Cc2ccccc2)CC(O)...,5076.0,-0.70,BBB+,InChI=1S/C37H48N6O5S2/c1-24(2)33(42-36(46)43(5...,NaN,R25|,...,10.505533,102.893937,720.312761,7.350130,10769,69,248,280,17.166667,11.138889
8,223,ribavirin,"1-[3,4-dihydroxy-5-(hydroxymethyl)oxolan-2-yl]...",NC(=O)c1ncn(C2OC(CO)C(O)C2O)n1,5064.0,-0.67,BBB+,InChI=1S/C8H12N4O5/c9-6(16)7-10-2-12(11-7)8-5(...,NaN,R2|R2|R8|R21|R25|R27|R46|R47|R40|,...,9.696279,65.758522,244.080769,8.416578,515,25,88,105,7.027778,3.805556
9,258,chembl195725,oxolan-3-yl n-[4-[(4-aminophenyl)sulfonyl-(2-m...,CC(C)CN(CC(O)C(Cc1ccccc1)NC(=O)OC1CCOC1)S(=O)(...,2177.0,-0.56,BBB+,"InChI=1S/C25H35N3O6S/c1-18(2)15-28(35(31,32)22...",NaN,R4|R40|R43|R47|,...,10.299407,84.150011,505.224657,7.217495,3813,50,176,200,12.562500,7.694444


In [14]:
avd_b3db.to_csv('B3DB_antiviral.tsv', sep='\t', index=Non
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                